In [1]:
from neon.layers import GeneralizedCost, LSTM, Affine
from neon.models import Model
from neon.optimizers import RMSProp
from neon.transforms import Logistic, Tanh, Softmax, CrossEntropyMulti
from neon.callbacks.callbacks import Callbacks
from neon.util.argparser import NeonArgparser
from neon.data import Text
from neon.initializers import GlorotUniform, Uniform, Orthonormal

In [2]:
import numpy as np
import os
import datetime
import cPickle
import sklearn
from sklearn.cross_validation import train_test_split

In [4]:
from neon.backends import gen_backend
be = gen_backend(backend='cpu',batch_size=20)
rnn_size = 256

In [5]:
from neon.data import Text

path = '/Users/tgn/Desktop/pilot/Benchmarks/P3B2/raw_text.txt'
data = Text(20, path)

In [8]:
init_glorot = GlorotUniform()
init_uniform = Uniform(-0.1/128, 0.1/128)
init_orthonormal = Orthonormal()

In [10]:
layers = [LSTM(output_size=rnn_size, init=init_glorot, activation=Tanh(),gate_activation=Tanh(),
               init_inner=init_orthonormal),
         Affine(len(data.vocab), init=init_glorot, activation=Softmax())]

In [11]:
optimizer = RMSProp()
cost = GeneralizedCost(costfunc=CrossEntropyMulti(usebits=True))

In [18]:
from neon.callbacks import Callbacks
from neon.models import Model
model = Model(layers=layers)
num_epochs = 2
fname = "neon_lstm_model"
model = Model(layers=layers)
callbacks = Callbacks(model, eval_set=data, eval_freq=num_epochs,
                      serialize=1, save_path=fname + '.pickle')

In [ ]:
model.fit(data, optimizer=optimizer, num_epochs=64, cost=cost, callbacks=callbacks)

Epoch 0   [Train |                    |   80/7177 batches, 4.76 cost, 4.49s]

//anaconda/lib/python2.7/site-packages/neon-1.8.2-py2.7.egg/neon/backends/nervanacpu.py:486: RuntimeWarning: overflow encountered in square
  "sqr": lambda left: np.square(left),


Epoch 0   [Train |                    |  186/7177 batches, nan cost, 10.31s]

//anaconda/lib/python2.7/site-packages/neon-1.8.2-py2.7.egg/neon/backends/nervanacpu.py:499: RuntimeWarning: overflow encountered in add
  "add": lambda left, right: left + right,
//anaconda/lib/python2.7/site-packages/neon-1.8.2-py2.7.egg/neon/backends/nervanacpu.py:501: RuntimeWarning: invalid value encountered in multiply
  "mul": lambda left, right: left * right,


Epoch 0   [Train |█                   |  532/7177 batches, nan cost, 28.96s]

In [67]:
a = np.arange(60).reshape((10, 3, 2))